[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/semantic-search/sparse/splade/splade-vector-generation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/semantic-search/sparse/splade/splade-vector-generation.ipynb)

# SPLADE Sparse-Dense Embedding Generation

[![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/fast-link.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-quora.ipynb)

## Overview

[SPLADE](https://www.pinecone.io/learn/splade/) is a class of models that produce sparse embeddings. Dense embeddings are often difficult to interpret, but sparse embeddings have clearly identifiable token overlap, making sparse vector search results more interpretable. SPLADE models have been shown to consistently outperform dense models, particularly in out-of-domain settings. 

The following guide will show you how to construct SPLADE embeddings to use in Pinecone's sparse-dense vectors. See the [companion guide to skip embedding generation](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-quora.ipynb).

## Prerequisites

We'll install the required libraries:

In [16]:
!pip install -qU \
    pinecone-text[splade] \
    pandas==2.0.2 \
    datasets==2.12.0 \
    pyarrow

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from tqdm.notebook import tqdm

## Quora Dataset

First We'll load the popular [Quora dataset](https://huggingface.co/datasets/quora).
The Quora dataset is composed of question pairs, and the task is to determine if the questions are duplications of each other (have the same meaning).

In [18]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("quora")

Found cached dataset quora (/Users/amnoncatav/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04)


  0%|          | 0/1 [00:00<?, ?it/s]

Let's look on a single record in this dataset

In [19]:
raw_df = dataset["train"].to_pandas()

raw_df.iloc[7].to_dict()

{'questions': {'id': array([15, 16], dtype=int32),
  'text': array(['How can I be a good geologist?',
         'What should I do to be a great geologist?'], dtype=object)},
 'is_duplicate': True}

As you can see above, each record contains two questions and a binary label that indicates whether the questions duplicates of each other.

Before we start to create embedding from our data, we need to process it. First, we need to convert the JSON column into distinct columns within our dataframe.

In [20]:
raw_df["q1"] = raw_df["questions"].apply(lambda q: q["text"][0])
raw_df["q2"] = raw_df["questions"].apply(lambda q: q["text"][1])
raw_df["id1"] = raw_df["questions"].apply(lambda q: q["id"][0])
raw_df["id2"] = raw_df["questions"].apply(lambda q: q["id"][1])

q1_to_q2 = raw_df.copy().rename(columns={"q1": "text", "id1": "id", "id2": "dq_id"}).drop(columns=["questions", "q2"])
q2_to_q1 = raw_df.copy().rename(columns={"q2": "text", "id2": "id", "id1": "dq_id"}).drop(columns=["questions", "q1"])
flat_df = pd.concat([q1_to_q2, q2_to_q1])

flat_df.head()

,is_duplicate,text,id,dq_id
0,False,What is the step by step guide to invest in sh...,1,2
1,False,What is the story of Kohinoor (Koh-i-Noor) Dia...,3,4
2,False,How can I increase the speed of my internet co...,5,6
3,False,Why am I mentally very lonely? How can I solve...,7,8
4,False,"Which one dissolve in water quikly sugar, salt...",9,10


Next, we take a sample of the data and retaining just one row per question while storing a list of duplicated question IDs for each question.

In [21]:
df = flat_df.drop_duplicates("id").head(2000)
df["duplicated_questions"] = df["id"].apply(lambda qid: flat_df[flat_df["id"] == qid]["dq_id"].tolist())
df = df.drop(columns=["dq_id", "is_duplicate"])

df.tail()

,text,id,duplicated_questions
2003,What name will you give to a mathematics educa...,3985,"[3986, 535562]"
2004,What universities does Federal-Mogul recruit n...,3987,"[3988, 23638]"
2005,Why do people go for mba after masters in engi...,3989,"[3990, 161829, 316113, 168810]"
2006,How does a facebook account get hacked?,3991,"[3992, 379602, 35699]"
2007,Why do black people get upset when non-black r...,3993,[3994]


### Sparse Embeddings with SPLADE 

In the following example we will use the [naver/splade-cocondenser-ensembledistil](https://huggingface.co/naver/splade-cocondenser-ensembledistil) SPLADE model.


In [22]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"running on {device}")

running on cpu


SPLADE is currently avialble in Hugging Face only as a BERT-based transformer. To encode text into sparse vectors, simple modifications to the transformer's output are required, as detailed in the [paper](https://arxiv.org/abs/2109.10086). For ease of use, we have encapsulated this model in the `pinecone-text` library.

In [23]:
from pinecone_text.sparse import SpladeEncoder

splade = SpladeEncoder(device=device)

In [24]:
doc = "what is the capital of france?"
sparse_vector = splade.encode_documents(doc)


### Dense Model

We use the popular all-MiniLM-L6-v2 model available on Hugging Face for dense vectors.

In [25]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    'sentence-transformers/all-MiniLM-L6-v2',
    device=device
)

### Compute Dense & Sparse Embeddings

Create BM25 sparse embeddings:

In [26]:
sparse_values = []

batch_size = 20
for i in tqdm(range(0, len(df), batch_size)):
  sparse_values += splade.encode_documents(df.iloc[i:i + batch_size]["text"].tolist())

df['sparse_values'] = sparse_values

  0%|          | 0/100 [00:00<?, ?it/s]

And now encode dense vector embeddings:

In [27]:
batch_size = 20
dense_values = []
for i in tqdm(range(0, len(df), batch_size)):
  dense_values += model.encode(df.iloc[i:i + batch_size]["text"].tolist()).tolist()

df['values'] = dense_values

  0%|          | 0/100 [00:00<?, ?it/s]

We organize our dataframe to align to the `pinecone-datasets` format:

In [28]:
df_result = df.copy()
df_result["metadata"] = None
df_result["blob"] = df_result["text"].apply(lambda t: {"text": t})
df_result = df_result.drop(columns="text")

In [29]:
df_result.head()

,id,duplicated_questions,sparse_values,values,metadata,blob
0,1,[2],"{'indices': [1012, 2011, 2017, 2022, 2044, 219...","[0.06814990937709808, -0.03966414928436279, -0...",None,{'text': 'What is the step by step guide to in...
1,3,"[4, 282170, 380197, 488853]","{'indices': [1005, 1006, 1007, 1011, 1012, 104...","[-0.04679807275533676, 0.15511494874954224, -0...",None,{'text': 'What is the story of Kohinoor (Koh-i...
2,5,[6],"{'indices': [2017, 2064, 2076, 2078, 2096, 211...","[-0.02832488901913166, 0.037209585309028625, -...",None,{'text': 'How can I increase the speed of my i...
3,7,[8],"{'indices': [1045, 1998, 2009, 2017, 2033, 204...","[0.06325336545705795, -0.05639313533902168, 0....",None,{'text': 'Why am I mentally very lonely? How c...
4,9,"[10, 109465, 480204]","{'indices': [1012, 1999, 2028, 2029, 2135, 224...","[-0.04876847192645073, -0.025538966059684753, ...",None,{'text': 'Which one dissolve in water quikly s...


And now we have all we need to start using Pinecone vector database 🚀

For more details on that, check out [this notebook](https://github.com/pinecone-io/examples/blob/master/pinecone/sparse/splade/splade-quora.ipynb).